In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'RGAPI-21d97aaa-2936-48f8-8d14-1b7ec8e11835'

# 본인 소환사명입니다.
summoner_name = 'bbo bbo'

In [3]:
# SUMMONER-V4 API URL
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key

In [4]:
r = requests.get(requesturl)

In [5]:
# 불러온 데이터 확인
r.json()

{'id': 'DSE-Ry3Pe7XnkKLRszZWggYpUhzPKoSRoztg4zy4LTI-aw',
 'accountId': '9SABmCP35rDHhLJFB4kcLRHjUhAcFWreOjTHQSYbLpq4',
 'puuid': 'IAFEULvrkOAY3c7Q4FZMoseecg2SLCjvVNg5x2GX_sPQgczmrdlWMHwuN5gdj1ficZQzWd2bqyhk5g',
 'name': 'bbo bbo',
 'profileIconId': 21,
 'revisionDate': 1633953827000,
 'summonerLevel': 95}

In [6]:
# 본인 puuid입니다.
summoner_puuid = r.json()['puuid']

In [7]:
# 불러오고 싶은 게임 수
# 100 이하로 하기
GAME_COUNT = 20

In [8]:
# MATCH-V5 API URL
requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+\
'/ids?start=0&count='+str(GAME_COUNT)+'&api_key='+api_key

In [9]:
req = requests.get(requesturl)

In [10]:
# 불러온 데이터 확인
req.json()

['KR_5506895682',
 'KR_5506913637',
 'KR_5506891328',
 'KR_5506847812',
 'KR_5506824180',
 'KR_5506709463',
 'KR_5502160829',
 'KR_5478792575',
 'KR_5478790037',
 'KR_5478686728',
 'KR_5467267693',
 'KR_5467126483',
 'KR_5467087581',
 'KR_5467012344',
 'KR_5466942844',
 'KR_5466816785',
 'KR_5466619212',
 'KR_5466512933',
 'KR_5466434635',
 'KR_5466430185']

In [18]:
cnt = 0
special_mode = 0
blue_win = 0

for i in range(GAME_COUNT):
    matchId = req.json()[i]
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/'+matchId\
    +'?api_key='+api_key
    r = requests.get(requesturl)
    
    pd.set_option('display.max_columns',None)
    df = pd.DataFrame(r.json()['info']['teams'])
    
    team1_visionScore = 0
    for j in range(0, 5):
        team1_visionScore += (r.json()['info']['participants'][j]['visionScore'])
    team2_visionScore = 0
    for j in range(5, 10):
        team2_visionScore += (r.json()['info']['participants'][j]['visionScore'])
        
    print(f"\n=========[game{i+1}]========")
    
    if r.json()['info']['gameMode'] != "CLASSIC":
        print("특별 게임 모드")
        special_mode += 1
        continue
        
    df['teamVisionScore'] = [team1_visionScore, team2_visionScore]
    COLUMNS = ['win', 'teamVisionScore']
    print(df[COLUMNS])
    
    if r.json()['info']['teams'][0]['win'] == True:
        blue_win += 1
        if team1_visionScore > team2_visionScore:
            cnt += 1
    if r.json()['info']['teams'][1]['win'] == True and team2_visionScore > team1_visionScore: 
        cnt += 1
        
print("\n\n=======================총 정리====================\n")
print(f"총 표본 : {GAME_COUNT}, 특별 게임 모드 : {special_mode}")
print(f"시야점수가 높은 팀이 이길 확률 : {cnt}/{GAME_COUNT-special_mode}", end = "")
print("   ({0:.2f}%)".format(cnt/(GAME_COUNT-special_mode)*100))
print(f"블루팀이 이길 확률 : {blue_win}/{GAME_COUNT-special_mode}", end = "")
print("   ({0:.2f}%)".format(blue_win/(GAME_COUNT-special_mode)*100))


=========[game1]========
     win  teamVisionScore
0  False              155
1   True              224

=========[game2]========
     win  teamVisionScore
0  False              151
1   True              201

=========[game3]========
     win  teamVisionScore
0  False              182
1   True              176

=========[game4]========
     win  teamVisionScore
0   True              102
1  False               80

=========[game5]========
     win  teamVisionScore
0  False              231
1   True              250

=========[game6]========
     win  teamVisionScore
0   True              131
1  False               97

=========[game7]========
특별 게임 모드(우르프, 칼바람 등)

=========[game8]========
특별 게임 모드(우르프, 칼바람 등)

=========[game9]========
특별 게임 모드(우르프, 칼바람 등)

=========[game10]========
특별 게임 모드(우르프, 칼바람 등)

=========[game11]========
     win  teamVisionScore
0   True              234
1  False              216

=========[game12]========
     win  teamVisionScore
0   True              100
1  